In [3]:
import pandas as pd
import numpy as np

import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.abspath(''))))
from yumspeak_ml.params import *

import geocoder

In [2]:
b1_r_df = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/batch1_restaurants_dataset.csv').drop(columns=['Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'])
b1_r_df.head().T

,0,1,2,3,4
place_id,ChIJ8Y3ZDL8R2jERZdF0_9U5XSo,ChIJuXEcL-gX2jEReUwbpfL8C4c,ChIJebVvXAAZ2jERRlzs4R2Qsm4,ChIJLSQEyKwZ2jERsJYlxkDMr1g,ChIJLTGN8D4Z2jERP4VNFXTnIpQ
name,(Fu Chan) Harummanis Muslim Food,#01-22 Wufu Hakka Noodle,116 PHỐ ĐÊM,123 ZÔ - Ẩm Thực Việt,123 Zô The Việt Kitchen
reviews,35,1,3,252,19
main_category,Malaysian restaurant,Noodle shop,Vietnamese restaurant,Restaurant,Vietnamese restaurant
categories,['Malaysian restaurant'],['Noodle shop'],['Vietnamese restaurant'],['Restaurant'],['Vietnamese restaurant']
main_rating,2.1,5.0,5.0,4.4,3.7
address,"145 Teck Whye Ln, Singapore 680145","590 Upper Thomson Rd, #01-22, Singapore 574419","116 Mackenzie Rd, Singapore 228710","12 Gopeng St, #01-45, 49-51 icon village, Sing...","18 Raffles Quay, #01-48 Lau Pa Sat, Singapore ..."
link,https://www.google.com/maps/place/(Fu+Chan)+Ha...,https://www.google.com/maps/place/%2301-22+Wuf...,https://www.google.com/maps/place/116+PH%E1%BB...,https://www.google.com/maps/place/123+Z%C3%94+...,https://www.google.com/maps/place/123+Z%C3%B4+...
review_photos,['https://lh5.googleusercontent.com/p/AF1QipNq...,[],['https://lh5.googleusercontent.com/p/AF1QipP1...,['https://lh5.googleusercontent.com/p/AF1QipNM...,['https://lh5.googleusercontent.com/p/AF1QipNI...


### Drop Duplicates

In [3]:
print(b1_r_df.shape)
b1_r_df.duplicated().sum()

(1356, 9)


173

In [4]:
b1_r_df = b1_r_df.drop_duplicates(subset=['place_id', 'name', 'reviews', 'address']).copy()
b1_r_df.shape

(1134, 9)

In [5]:
cats_to_remove = [cat.lower() for cat in CATS_TO_REMOVE]
mask = b1_r_df['main_category'].str.lower().isin(cats_to_remove)
b1_r_df = b1_r_df[~mask].copy()

In [6]:
b1_r_df = b1_r_df[~b1_r_df['name'].isin(NAME_TO_DROP)]

NameError: name 'NAME_TO_DROP' is not defined

In [7]:
b1_r_df.shape

(804, 9)

In [8]:
b1_r_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 0 to 1355
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       804 non-null    object 
 1   name           804 non-null    object 
 2   reviews        804 non-null    int64  
 3   main_category  785 non-null    object 
 4   categories     785 non-null    object 
 5   main_rating    804 non-null    float64
 6   address        796 non-null    object 
 7   link           804 non-null    object 
 8   review_photos  804 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 62.8+ KB


In [9]:
# Fill na in main_category as unknown
b1_r_df['main_category'].fillna('unknown', inplace=True)
b1_r_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 0 to 1355
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       804 non-null    object 
 1   name           804 non-null    object 
 2   reviews        804 non-null    int64  
 3   main_category  804 non-null    object 
 4   categories     785 non-null    object 
 5   main_rating    804 non-null    float64
 6   address        796 non-null    object 
 7   link           804 non-null    object 
 8   review_photos  804 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 62.8+ KB


In [10]:
# Fill na in categories as unknown
b1_r_df['categories'] = b1_r_df['categories'].fillna("['unknown']").apply(eval)

In [11]:
b1_r_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 0 to 1355
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       804 non-null    object 
 1   name           804 non-null    object 
 2   reviews        804 non-null    int64  
 3   main_category  804 non-null    object 
 4   categories     804 non-null    object 
 5   main_rating    804 non-null    float64
 6   address        796 non-null    object 
 7   link           804 non-null    object 
 8   review_photos  804 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 62.8+ KB


### Generate Coordinates lat lng

In [12]:
import re

def find_coordinates(link):
    match = re.search('!3d(-?\d+(?:\.\d+)?)!4d(-?\d+(?:\.\d+))', link)
    coordinates = [float(match.group(1)), float(match.group(2))]
    return coordinates

In [13]:
def add_lat_lng(df):
    df['coordinates'] = df['link'].apply(lambda x: find_coordinates(x))
    df['latitude'] = df['coordinates'].apply(lambda x: x[0])
    df['longtitude'] = df['coordinates'].apply(lambda x: x[1])
    return df

In [14]:
b1_r_df = add_lat_lng(b1_r_df)
b1_r_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804 entries, 0 to 1355
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       804 non-null    object 
 1   name           804 non-null    object 
 2   reviews        804 non-null    int64  
 3   main_category  804 non-null    object 
 4   categories     804 non-null    object 
 5   main_rating    804 non-null    float64
 6   address        796 non-null    object 
 7   link           804 non-null    object 
 8   review_photos  804 non-null    object 
 9   coordinates    804 non-null    object 
 10  latitude       804 non-null    float64
 11  longtitude     804 non-null    float64
dtypes: float64(3), int64(1), object(8)
memory usage: 81.7+ KB


In [15]:
# Singapore lat lng borders
# lat 1.129 1.493
# lng 103.557 104.131

b1_r_df = b1_r_df[b1_r_df['latitude'].between(left=1.129, right=1.493)]
b1_r_df = b1_r_df[b1_r_df['longtitude'].between(left=103.557, right=104.131)]
b1_r_df = b1_r_df.reset_index(drop=True)
print(b1_r_df.shape)
b1_r_df.head()

(791, 12)


,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude
0,ChIJ8Y3ZDL8R2jERZdF0_9U5XSo,(Fu Chan) Harummanis Muslim Food,35,Malaysian restaurant,[Malaysian restaurant],2.1,"145 Teck Whye Ln, Singapore 680145",https://www.google.com/maps/place/(Fu+Chan)+Ha...,['https://lh5.googleusercontent.com/p/AF1QipNq...,"[1.3813371, 103.7524994]",1.381337,103.752499
1,ChIJuXEcL-gX2jEReUwbpfL8C4c,#01-22 Wufu Hakka Noodle,1,Noodle shop,[Noodle shop],5.0,"590 Upper Thomson Rd, #01-22, Singapore 574419",https://www.google.com/maps/place/%2301-22+Wuf...,[],"[1.3725339, 103.8290593]",1.372534,103.829059
2,ChIJebVvXAAZ2jERRlzs4R2Qsm4,116 PHỐ ĐÊM,3,Vietnamese restaurant,[Vietnamese restaurant],5.0,"116 Mackenzie Rd, Singapore 228710",https://www.google.com/maps/place/116+PH%E1%BB...,['https://lh5.googleusercontent.com/p/AF1QipP1...,"[1.3066408, 103.8472818]",1.306641,103.847282
3,ChIJLSQEyKwZ2jERsJYlxkDMr1g,123 ZÔ - Ẩm Thực Việt,252,Restaurant,[Restaurant],4.4,"12 Gopeng St, #01-45, 49-51 icon village, Sing...",https://www.google.com/maps/place/123+Z%C3%94+...,['https://lh5.googleusercontent.com/p/AF1QipNM...,"[1.2754032, 103.8445304]",1.275403,103.844530
4,ChIJLTGN8D4Z2jERP4VNFXTnIpQ,123 Zô The Việt Kitchen,19,Vietnamese restaurant,[Vietnamese restaurant],3.7,"18 Raffles Quay, #01-48 Lau Pa Sat, Singapore ...",https://www.google.com/maps/place/123+Z%C3%B4+...,['https://lh5.googleusercontent.com/p/AF1QipNI...,"[1.2807598, 103.8504144]",1.280760,103.850414


### Get postal / district code

In [16]:
b1_r_df[b1_r_df['address'].isna()].shape

(8, 12)

In [17]:
b1_r_df.coordinates.isna().sum()

0

In [18]:
coor_1 = b1_r_df.coordinates[0]
coor_1

[1.3813371, 103.7524994]

In [19]:
postal = geocoder.mapbox(coor_1, method='reverse', key=MAP_API).json['postal'][0:2]

In [20]:
# address column not na, find postal code. if error find postal code with mapbox
# not na find postal code with mapbox error return address
def get_postal_code(row):
    if isinstance(row['address'], str):
        try:
            match = re.search(r'\b\d{6}\b', row['address'])
            postal_code = match.group(0)
            row['postal_code'] = postal_code[:2]
            return row
        except:
            g =geocoder.mapbox(row['coordinates'], method='reverse', key=MAP_API)
            row['address'] = g.json['postal']
            row['postal_code'] = g.json['postal'][:2]
            return row
    else:
        try:
            g =geocoder.mapbox(row['coordinates'], method='reverse', key=MAP_API)
            row['address'] = g.json['postal']
            row['postal_code'] = g.json['postal'][:2]
            return row
        except:
            print(f'Error: {row['address']}')
            return row['address']

SyntaxError: f-string: unmatched '[' (1494344250.py, line 22)

In [21]:
b1_r_df = b1_r_df.apply(get_postal_code, axis=1)
b1_r_df.info()

NameError: name 'get_postal_code' is not defined

In [22]:
b1_r_df['address'].isna().sum()

8

In [23]:
b1_r_df['postal_code'].isna().sum()

KeyError: 'postal_code'

In [24]:
b1_r_df.postal_code.unique()

AttributeError: 'DataFrame' object has no attribute 'postal_code'

In [25]:
b1_r_df.to_csv('/Users/triciascy/code/scytricia/yumspeak/name_data/postal_restaurant_b1.csv',index=False, header=True)

OSError: Cannot save file into a non-existent directory: '/Users/triciascy/code/scytricia/yumspeak/name_data'

In [ ]:
#district
#region

In [26]:
postal_to_district = {
    '01': '01', '02': '01', '03': '01', '04': '01', '05': '01', '06': '01',
    '07': '02', '08': '02', '09': '04', '10': '04', '11': '05', '12': '05',
    '13': '05', '14': '03', '15': '03', '16': '03', '17': '06', '18': '07',
    '19': '07', '20': '08', '21': '08', '22': '09', '23': '09', '24': '10',
    '25': '10', '26': '10', '27': '10', '28': '11', '29': '11', '30': '11',
    '31': '12', '32': '12', '33': '12', '34': '13', '35': '13', '36': '13',
    '37': '13', '38': '14', '39': '14', '40': '14', '41': '14', '42': '15',
    '43': '15', '44': '15', '45': '15', '46': '16', '47': '16', '48': '16',
    '49': '17', '50': '17', '51': '18', '52': '18', '53': '19', '54': '19',
    '55': '19', '56': '20', '57': '20', '58': '21', '59': '21', '60': '22',
    '61': '22', '62': '22', '63': '22', '64': '22', '65': '23', '66': '23',
    '67': '23', '68': '23', '69': '24', '70': '24', '71': '24', '72': '25',
    '73': '25', '75': '27', '76': '27', '77': '26', '78': '26', '79': '28',
    '80': '28', '81': '17', '82': '19'
}

In [27]:
district_to_region = {
    '01': 'City', '02': 'City', '03': 'South', '04': 'South', '05': 'West', '06': 'City',
    '07': 'City', '08': 'Central', '09': 'Central', '10': 'Central', '11': 'Central', '12': 'Central',
    '13': 'East', '14': 'East', '15': 'East', '16': 'East', '17': 'East', '18': 'East',
    '19': 'North', '20': 'North', '21': 'West', '22': 'West', '23': 'West', '24': 'West',
    '25': 'North', '26': 'North', '27': 'North', '28': 'North'
}

In [28]:
b1_r_df['district_code'] = b1_r_df['postal_code'].map(postal_to_district)
b1_r_df.head()

KeyError: 'postal_code'

In [29]:
b1_r_df['region'] = b1_r_df['district_code'].map(district_to_region)
b1_r_df

KeyError: 'district_code'

In [30]:
posb1_r_dftal_b1['postal_code'].unique()

NameError: name 'posb1_r_dftal_b1' is not defined

In [31]:
b1_r_df.district_code.unique()

AttributeError: 'DataFrame' object has no attribute 'district_code'

In [32]:
b1_r_df.region.unique()

AttributeError: 'DataFrame' object has no attribute 'region'

In [33]:
final_b1 = b1_r_df.copy()
final_b1

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude
0,ChIJ8Y3ZDL8R2jERZdF0_9U5XSo,(Fu Chan) Harummanis Muslim Food,35,Malaysian restaurant,[Malaysian restaurant],2.1,"145 Teck Whye Ln, Singapore 680145",https://www.google.com/maps/place/(Fu+Chan)+Ha...,['https://lh5.googleusercontent.com/p/AF1QipNq...,"[1.3813371, 103.7524994]",1.381337,103.752499
1,ChIJuXEcL-gX2jEReUwbpfL8C4c,#01-22 Wufu Hakka Noodle,1,Noodle shop,[Noodle shop],5.0,"590 Upper Thomson Rd, #01-22, Singapore 574419",https://www.google.com/maps/place/%2301-22+Wuf...,[],"[1.3725339, 103.8290593]",1.372534,103.829059
2,ChIJebVvXAAZ2jERRlzs4R2Qsm4,116 PHỐ ĐÊM,3,Vietnamese restaurant,[Vietnamese restaurant],5.0,"116 Mackenzie Rd, Singapore 228710",https://www.google.com/maps/place/116+PH%E1%BB...,['https://lh5.googleusercontent.com/p/AF1QipP1...,"[1.3066408, 103.8472818]",1.306641,103.847282
3,ChIJLSQEyKwZ2jERsJYlxkDMr1g,123 ZÔ - Ẩm Thực Việt,252,Restaurant,[Restaurant],4.4,"12 Gopeng St, #01-45, 49-51 icon village, Sing...",https://www.google.com/maps/place/123+Z%C3%94+...,['https://lh5.googleusercontent.com/p/AF1QipNM...,"[1.2754032, 103.8445304]",1.275403,103.844530
4,ChIJLTGN8D4Z2jERP4VNFXTnIpQ,123 Zô The Việt Kitchen,19,Vietnamese restaurant,[Vietnamese restaurant],3.7,"18 Raffles Quay, #01-48 Lau Pa Sat, Singapore ...",https://www.google.com/maps/place/123+Z%C3%B4+...,['https://lh5.googleusercontent.com/p/AF1QipNI...,"[1.2807598, 103.8504144]",1.280760,103.850414
...,...,...,...,...,...,...,...,...,...,...,...,...
786,ChIJcWHhGKEZ2jERmOsLjIP6nnQ,Coffee byWassies,37,Art cafe,[Art cafe],4.9,"5 Hongkong St, Singapore 059648",https://www.google.com/maps/place/Coffee+byWas...,['https://lh5.googleusercontent.com/p/AF1QipOt...,"[1.2874767, 103.8478706]",1.287477,103.847871
787,ChIJK9FuLKEZ2jERZYDXi6XtZ2k,Coleman's Cafe,54,Cafe,[Cafe],3.7,"3 Coleman St, Singapore 179805",https://www.google.com/maps/place/Coleman's+Ca...,['https://lh5.googleusercontent.com/p/AF1QipPa...,"[1.2918776, 103.8500267]",1.291878,103.850027
788,ChIJ02kZpakZ2jERDQVTq10I8b0,Colony,2078,Restaurant,"[Restaurant, Buffet restaurant]",4.6,"7 Raffles Ave., Singapore 039799",https://www.google.com/maps/place/Colony/data=...,['https://lh5.googleusercontent.com/p/AF1QipMF...,"[1.2908195, 103.86009]",1.290820,103.860090
789,ChIJKdihA68Z2jER07IZ_I3hEVI,Colony Capital Asia Pacific Pte Ltd,0,unknown,[unknown],0.0,NaN,https://www.google.com/maps/place/Colony+Capit...,[],"[1.2956745, 103.8589973]",1.295675,103.858997


In [34]:
final_b1.to_csv('/Users/triciascy/code/scytricia/yumspeak/name_data/final_restaurant_b1.csv', index=False, header=True)

OSError: Cannot save file into a non-existent directory: '/Users/triciascy/code/scytricia/yumspeak/name_data'

In [ ]:
test = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/name_data/final_restaurant_b1.csv', dtype={'postal_code':str, 'district_code':str})
test

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude,postal_code,district_code,region
0,ChIJ8Y3ZDL8R2jERZdF0_9U5XSo,(Fu Chan) Harummanis Muslim Food,35,Malaysian restaurant,['Malaysian restaurant'],2.1,"145 Teck Whye Ln, Singapore 680145",https://www.google.com/maps/place/(Fu+Chan)+Ha...,['https://lh5.googleusercontent.com/p/AF1QipNq...,"[1.3813371, 103.7524994]",1.381337,103.752499,68,23,West
1,ChIJuXEcL-gX2jEReUwbpfL8C4c,#01-22 Wufu Hakka Noodle,1,Noodle shop,['Noodle shop'],5.0,"590 Upper Thomson Rd, #01-22, Singapore 574419",https://www.google.com/maps/place/%2301-22+Wuf...,[],"[1.3725339, 103.8290593]",1.372534,103.829059,57,20,North
2,ChIJebVvXAAZ2jERRlzs4R2Qsm4,116 PHỐ ĐÊM,3,Vietnamese restaurant,['Vietnamese restaurant'],5.0,"116 Mackenzie Rd, Singapore 228710",https://www.google.com/maps/place/116+PH%E1%BB...,['https://lh5.googleusercontent.com/p/AF1QipP1...,"[1.3066408, 103.8472818]",1.306641,103.847282,22,09,Central
3,ChIJLSQEyKwZ2jERsJYlxkDMr1g,123 ZÔ - Ẩm Thực Việt,252,Restaurant,['Restaurant'],4.4,"12 Gopeng St, #01-45, 49-51 icon village, Sing...",https://www.google.com/maps/place/123+Z%C3%94+...,['https://lh5.googleusercontent.com/p/AF1QipNM...,"[1.2754032, 103.8445304]",1.275403,103.844530,07,02,City
4,ChIJLTGN8D4Z2jERP4VNFXTnIpQ,123 Zô The Việt Kitchen,19,Vietnamese restaurant,['Vietnamese restaurant'],3.7,"18 Raffles Quay, #01-48 Lau Pa Sat, Singapore ...",https://www.google.com/maps/place/123+Z%C3%B4+...,['https://lh5.googleusercontent.com/p/AF1QipNI...,"[1.2807598, 103.8504144]",1.280760,103.850414,04,01,City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,ChIJK9FuLKEZ2jERZYDXi6XtZ2k,Coleman's Cafe,54,Cafe,['Cafe'],3.7,"3 Coleman St, Singapore 179805",https://www.google.com/maps/place/Coleman's+Ca...,['https://lh5.googleusercontent.com/p/AF1QipPa...,"[1.2918776, 103.8500267]",1.291878,103.850027,17,06,City
863,ChIJ02kZpakZ2jERDQVTq10I8b0,Colony,2078,Restaurant,"['Restaurant', 'Buffet restaurant']",4.6,"7 Raffles Ave., Singapore 039799",https://www.google.com/maps/place/Colony/data=...,['https://lh5.googleusercontent.com/p/AF1QipMF...,"[1.2908195, 103.86009]",1.290820,103.860090,03,01,City
864,ChIJKdihA68Z2jER07IZ_I3hEVI,Colony Capital Asia Pacific Pte Ltd,0,unknown,['unknown'],0.0,038989,https://www.google.com/maps/place/Colony+Capit...,[],"[1.2956745, 103.8589973]",1.295675,103.858997,03,01,City
865,ChIJ0eEWGp8Z2jERU0GVV6u922I,Dutch Colony Coffee Co.,382,Coffee shop,['Coffee shop'],4.2,"81 Clemenceau Ave, #01-21, Singapore 239917",https://www.google.com/maps/place/Dutch+Colony...,['https://lh5.googleusercontent.com/p/AF1QipMJ...,"[1.2922763, 103.8425461]",1.292276,103.842546,23,09,Central


In [35]:
df = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/cleaned_all_restaurants_dataset.csv', dtype={'full_postal_code': str, 'postal_code':str, 'district_code':str})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2706 entries, 0 to 2705
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   place_id          2706 non-null   object 
 1   name              2706 non-null   object 
 2   reviews           2706 non-null   int64  
 3   main_category     2706 non-null   object 
 4   main_rating       2706 non-null   float64
 5   address           2706 non-null   object 
 6   link              2706 non-null   object 
 7   review_photos     2706 non-null   object 
 8   cuisine           2706 non-null   object 
 9   latitude          2706 non-null   float64
 10  longtitude        2706 non-null   float64
 11  full_postal_code  2706 non-null   object 
 12  postal_code       2706 non-null   object 
 13  district_code     2706 non-null   object 
 14  region            2706 non-null   object 
dtypes: float64(3), int64(1), object(11)
memory usage: 317.2+ KB


In [36]:
df = df.dropna()

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2706 entries, 0 to 2705
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   place_id          2706 non-null   object 
 1   name              2706 non-null   object 
 2   reviews           2706 non-null   int64  
 3   main_category     2706 non-null   object 
 4   main_rating       2706 non-null   float64
 5   address           2706 non-null   object 
 6   link              2706 non-null   object 
 7   review_photos     2706 non-null   object 
 8   cuisine           2706 non-null   object 
 9   latitude          2706 non-null   float64
 10  longtitude        2706 non-null   float64
 11  full_postal_code  2706 non-null   object 
 12  postal_code       2706 non-null   object 
 13  district_code     2706 non-null   object 
 14  region            2706 non-null   object 
dtypes: float64(3), int64(1), object(11)
memory usage: 317.2+ KB


## Map of all restaurants in SG

In [4]:
# load merged dataset
df = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/merged_dataset.csv', low_memory=False, dtype={'full_postal_code': str, 'postal_code':str, 'district_code':str})

In [5]:
restaurant = df.drop_duplicates(subset=['place_id', 'name', 'address', 'latitude', 'longtitude']).reset_index(drop=True)
restaurant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2656 entries, 0 to 2655
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   place_id                             2656 non-null   object 
 1   name                                 2656 non-null   object 
 2   reviews                              2656 non-null   int64  
 3   main_category                        2656 non-null   object 
 4   main_rating                          2656 non-null   float64
 5   address                              2656 non-null   object 
 6   link                                 2656 non-null   object 
 7   review_photos                        2656 non-null   object 
 8   cuisine                              2656 non-null   object 
 9   latitude                             2656 non-null   float64
 10  longtitude                           2656 non-null   float64
 11  full_postal_code              

In [44]:
restaurant = restaurant[~(restaurant['full_postal_code'].str.len() < 6)]

In [45]:
grouped_data = restaurant.groupby(['place_id', "name", 'address', 'latitude', 'longtitude']).agg(
    count=("place_id", "count"),
).reset_index()

In [46]:
grouped_data

,place_id,name,address,latitude,longtitude,count
0,ChIJ-0DAQI4a2jERVIPajSdgj58,Sichuan Tianfu Restaurant 天府川菜 (Clementi),"3151 Commonwealth Ave W, #01-17/18 Grantral Ma...",1.314285,103.765086,1
1,ChIJ-131x5YZ2jEROumyYjpVEIU,Beauty in The Pot 美滋锅 at The Centrepoint,"176 Orchard Rd, #05-16 The Centrepoint, Singap...",1.301980,103.839695,1
2,ChIJ-44uRgoZ2jERAcE9_ZekQ8I,New Shah Alam Restaurant (Halal),"20-21 Circular Rd, Singapore 049376",1.287312,103.848949,1
3,ChIJ-4qafdMZ2jERomZxE3l1IFk,Cultivate Cafe,"2 Cook St, Maxwell Reserve, Singapore 078857",1.279044,103.844428,1
4,ChIJ-5a9fIoQ2jERGAUbBoxL2xk,Gayageum Korean Family Restaurant,"170 Upper Bukit Timah Road B2, #14 Shopping Ce...",1.343235,103.775923,1
...,...,...,...,...,...,...
2639,ChIJzeQ8gw0Z2jERhGLaINksUXI,Common Man Coffee Roasters - Stanley Street,"11, 12 Stanley St, 068730",1.280290,103.847970,1
2640,ChIJzw2mcnEZ2jERlMZG0uYKzqE,Lavi Taco,"195 Pearl's Hill Terrace, #01-54, Singapore 16...",1.284083,103.841548,1
2641,ChIJzxfdxiM92jERPBeI4yiW3ns,Shabu Sai @ Tampines 1,"10 Tampines Central 1, #02-04, Singapore 529536",1.354013,103.945327,1
2642,ChIJzz0bjg0W2jERWmZpdBnlHgY,Shihlin Taiwan Street Snacks,"1 Sengkang Square, #B1-51, Singapore 545078",1.392174,103.895032,1


In [47]:
import plotly.express as px

fig = px.scatter_mapbox(
    grouped_data,
    lat="latitude",
    lon="longtitude",
    hover_name="name",
    # size="count",
    # color="count",
    labels={
        "latitude": "Latitude",
        "longitude": "Longitude",
        # "count": "Number of Restaurants",
    },
    zoom=10,
    center={"lat": 1.3521, "lon": 103.8198},
    mapbox_style="streets",
    size_max=5
)

fig.update_layout(
    mapbox=dict(
        accesstoken=MAP_API,
        bearing=0,
        pitch=0,
    ),
    title={
        'text': "Restaurant Count by City in Singapore",
        'x': 0.1,
        'xanchor': 'center',
    },

    showlegend=False
)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': '<b>%{hovertext}</b><br><br>Latitude=%{lat}<br>longtitude=%{lon}<extra></extra>',
              'hovertext': array(['Sichuan Tianfu Restaurant 天府川菜 (Clementi)',
                                  'Beauty in The Pot 美滋锅 at The Centrepoint',
                                  'New Shah Alam Restaurant (Halal)', ..., 'Shabu Sai @ Tampines 1',
                                  'Shihlin Taiwan Street Snacks', 'Cherki'], dtype=object),
              'lat': array([1.3142853, 1.3019805, 1.2873123, ..., 1.3540125, 1.3921738, 1.275766 ]),
              'legendgroup': '',
              'lon': array([103.7650862, 103.839695 , 103.8489491, ..., 103.945327 , 103.895032 ,
                            103.8496856]),
              'marker': {'color': '#636efa'},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'subplot': 'mapbox',
              'type': 'scattermapbox'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'mapbox': {'accesstoken': ('pk.eyJ1Ijoic2N5dHJpY2lhIiwiYSI' ... '3NncifQ.8VAv3GfXjwYFPL-85v7Mwg'),
                          'bearing': 0,
                          'center': {'lat': 1.3521, 'lon': 103.8198},
                          'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
                          'pitch': 0,
                          'style': 'streets',
                          'zoom': 10},
               'margin': {'t': 60},
               'showlegend': False,
               'template': '...',
               'title': {'text': 'Restaurant Count by City in Singapore', 'x': 0.1, 'xanchor': 'center'}}
})

In [6]:
df[df.place_id=='ChIJzz0bjg0W2jERWmZpdBnlHgY']

,place_id,name,reviews,main_category,main_rating,address,link,review_photos,cuisine,latitude,...,district_code,region,rating,review_text,published_at_date,review_likes_count,response_from_owner_text,total_number_of_reviews_by_reviewer,total_number_of_photos_by_reviewer,is_local_guide
32201,ChIJzz0bjg0W2jERWmZpdBnlHgY,Shihlin Taiwan Street Snacks,191,Taiwanese,4.0,"1 Sengkang Square, #B1-51, Singapore 545078",https://www.google.com/maps/place/Shihlin+Taiw...,['https://lh5.googleusercontent.com/p/AF1QipOP...,Asian,1.392174,...,19,North,2,chicken was very dry and have a weird taste ve...,2024-08-23,0,"Dear Jesper Phua, Thank you for your feedback....",10.0,0.0,False
32202,ChIJzz0bjg0W2jERWmZpdBnlHgY,Shihlin Taiwan Street Snacks,191,Taiwanese,4.0,"1 Sengkang Square, #B1-51, Singapore 545078",https://www.google.com/maps/place/Shihlin+Taiw...,['https://lh5.googleusercontent.com/p/AF1QipOP...,Asian,1.392174,...,19,North,3,The portion of tempura was too little for larg...,2024-06-30,0,"Dear Terry, Thank you for your feedback. Quali...",3.0,0.0,False
32203,ChIJzz0bjg0W2jERWmZpdBnlHgY,Shihlin Taiwan Street Snacks,191,Taiwanese,4.0,"1 Sengkang Square, #B1-51, Singapore 545078",https://www.google.com/maps/place/Shihlin+Taiw...,['https://lh5.googleusercontent.com/p/AF1QipOP...,Asian,1.392174,...,19,North,1,Terrible service by a female stuff whom had at...,2024-06-30,1,"Dear Infinity Army, Thank you for your feedbac...",2.0,0.0,False
32204,ChIJzz0bjg0W2jERWmZpdBnlHgY,Shihlin Taiwan Street Snacks,191,Taiwanese,4.0,"1 Sengkang Square, #B1-51, Singapore 545078",https://www.google.com/maps/place/Shihlin+Taiw...,['https://lh5.googleusercontent.com/p/AF1QipOP...,Asian,1.392174,...,19,North,2,"Waited for food, it arrived. I held the bottom...",2024-05-30,0,"Dear Sir Ruffles, Thank you for your feedback....",16.0,3.0,False
32205,ChIJzz0bjg0W2jERWmZpdBnlHgY,Shihlin Taiwan Street Snacks,191,Taiwanese,4.0,"1 Sengkang Square, #B1-51, Singapore 545078",https://www.google.com/maps/place/Shihlin+Taiw...,['https://lh5.googleusercontent.com/p/AF1QipOP...,Asian,1.392174,...,19,North,5,Good service attitude. My favorite meat floss ...,2024-04-30,0,"Dear customer, Thank you for your kind encoura...",1.0,0.0,False
32206,ChIJzz0bjg0W2jERWmZpdBnlHgY,Shihlin Taiwan Street Snacks,191,Taiwanese,4.0,"1 Sengkang Square, #B1-51, Singapore 545078",https://www.google.com/maps/place/Shihlin+Taiw...,['https://lh5.googleusercontent.com/p/AF1QipOP...,Asian,1.392174,...,19,North,5,"Delicious food, fast",2024-04-30,0,NaN,5.0,1.0,False
32207,ChIJzz0bjg0W2jERWmZpdBnlHgY,Shihlin Taiwan Street Snacks,191,Taiwanese,4.0,"1 Sengkang Square, #B1-51, Singapore 545078",https://www.google.com/maps/place/Shihlin+Taiw...,['https://lh5.googleusercontent.com/p/AF1QipOP...,Asian,1.392174,...,19,North,5,This staff very friendly & patience.,2024-04-30,0,"Dear customer, Thank you for your kind encoura...",1.0,1.0,False
32208,ChIJzz0bjg0W2jERWmZpdBnlHgY,Shihlin Taiwan Street Snacks,191,Taiwanese,4.0,"1 Sengkang Square, #B1-51, Singapore 545078",https://www.google.com/maps/place/Shihlin+Taiw...,['https://lh5.googleusercontent.com/p/AF1QipOP...,Asian,1.392174,...,19,North,5,Kid-friendliness: Easy to eat Food,2024-04-30,0,"Dear customer, Thank you for your kind encoura...",1.0,1.0,False
32209,ChIJzz0bjg0W2jERWmZpdBnlHgY,Shihlin Taiwan Street Snacks,191,Taiwanese,4.0,"1 Sengkang Square, #B1-51, Singapore 545078",https://www.google.com/maps/place/Shihlin+Taiw...,['https://lh5.googleusercontent.com/p/AF1QipOP...,Asian,1.392174,...,19,North,5,Love the crispy chicken!,2024-01-30,0,"Dear customer, Thank you for your kind encoura...",670.0,1859.0,True
32210,ChIJzz0bjg0W2jERWmZpdBnlHgY,Shihlin Taiwan Street Snacks,191,Taiwanese,4.0,"1 Sengkang Square, #B1-51, Singapore 545078",https://www.google.com/maps/place/Shihlin+Taiw...,['https://lh5.googleusercontent.com/p/AF1QipOP...,Asian,1.392174,...,19,North,5,"Chicken chop is crispy, delicious and delicious",2023-11-30,0,"Dear customer, Thank you for your kind e